In [ ]:
# open Outlook using Python
#import os
#os.startfile("outlook")

# Close program using Python
# https://stackoverflow.com/questions/5625524/how-to-close-a-program-using-python
# https://www.codeforests.com/2020/07/04/python-close-windows-process/

In [ ]:
import win32com.client
import re
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import os
from datetime import datetime

In [ ]:
# https://docs.microsoft.com/en-us/office/vba/api/overview/outlook
# https://docs.microsoft.com/en-us/office/vba/outlook/concepts/forms/outlook-fields-and-equivalent-properties
# https://docs.microsoft.com/en-us/office/vba/outlook/concepts/forms/standard-fields-overview



outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
for account in outlook.Accounts:
    print(account.DeliveryStore.DisplayName)
    print(account)
    #print("Inbox", outlook.Folders.Item(str(account)).Folders.Item("Inbox").Items.Count)
    for folder in outlook.Folders.Item(str(account)).Folders:
        print(folder, outlook.Folders.Item(str(account)).Folders.Item(str(folder)).Items.Count)

In [ ]:
for account in outlook.Accounts:
    print(account.DeliveryStore.DisplayName)
    print(account)
    print("Inbox", outlook.Folders.Item(str(account)).Folders.Item("Inbox").Items.Count)
    inbox = outlook.Folders.Item(str(account)).Folders.Item("Inbox")
    msgs = inbox.Items
    
    # It seems to move (or delete) messages correctly they need to be done in reverse order.
    # https://stackoverflow.com/questions/53523943/python-win32com-delete-multiple-emails-in-outlook
   
    for msg in reversed(msgs):
        print(msg.Subject)
        #print(msg.SentOnBehalfOfName)
        #print(msg.To)
        #print(msg.Body)
        #print(msg.Importance)
        #print(msg.CreationTime)
        #print(msg.SentOn)
        #print(msg.ReceivedTime)
        #print(msg.LastModificationTime)
        #print(msg.Attachments)
        #print(msg.CC)
        #print(msg.BCC)
        #print(msg.ReplyRecipientNames)
        #print(msg.Parent)
        #print(msg.MessageClass)
        #print(msg.FlagStatus)
        #print(msg.Sensitivity)
        #print(msg.UnRead)
        #print(msg.Size)
        #print(msg.SenderEmailAddress)
        #print(msg.Recipients)
        
        # The command below will move the message to the Deleted Items folder.
        #msg.Move(folder.Folders.Item("Deleted Items"))



In [ ]:
for account in outlook.Accounts:
    print(account.DeliveryStore.DisplayName)
    print(account)
    print("Inbox", outlook.Folders.Item(str(account)).Folders.Item("Inbox").Items.Count)
    inbox = outlook.Folders.Item(str(account)).Folders.Item("Inbox")
    msgs = inbox.Items
    
    msg_counts = {}
    
    # It seems to move (or delete) messages correctly they need to be done in reverse order.
    # https://stackoverflow.com/questions/53523943/python-win32com-delete-multiple-emails-in-outlook
   
    for msg in reversed(msgs):
        
        if msg.Subject not in msg_counts:
            find = re.search('NCID Value :(.*)', msg.Subject)
            ncid = find.group(1) if find else "NCID not found"
            msg_counts[msg.Subject] = [msg.Body,1,ncid]
    else:
            msg_counts[msg.Subject][1] += 1

        
msg_counts 

# switch the dictionary for a dataframe before saving to Excel file
countsdf = pd.DataFrame.from_dict(msg_counts, orient = 'index')
countsdf.reset_index(inplace=True)
countsdf.columns = ['subject','message', 'count', 'NCID']
countsdf = countsdf.sort_values(by='count', ascending=False)
countsdf

In [ ]:
# get path for current user
homepath = os.getenv('USERPROFILE')

# get the current time to name file
now = datetime.now()

# YYYY-mm-dd-H-M-S
dt_string = now.strftime("%Y-%m-%d-%H-%M-%S")

# path for final report. We'll define an Excel writer object and the target file
Excelwriter = pd.ExcelWriter(homepath + '\\Desktop\\' + dt_string + '_testOutput.xlsx',engine="xlsxwriter")

countsdf.to_excel(Excelwriter, sheet_name="counts",index=False)

Excelwriter.save()

print("Done.")